primeiro baixar o dataset do [kaggle](https://www.kaggle.com/datasets/fredericods/ptbr-sentiment-analysis-datasets?select=b2w.csv)

In [ ]:
!pip install wordcloud
!pip install nltk
import nltk
nltk.download("all")

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

avaliacoes = pd.read_csv('https://media.githubusercontent.com/media/rivervaladao/FIAP-POS-TECH/master/IA_PLN/datasets/b2w.csv')
avaliacoes.describe()


,original_index,polarity,rating,kfold_polarity,kfold_rating
count,132373.000000,116058.000000,132373.000000,132373.000000,132373.000000
mean,66186.000000,0.691895,3.492004,4.698813,5.499921
std,38212.937929,0.461712,1.530289,3.434905,2.872311
min,0.000000,0.000000,1.000000,-1.000000,1.000000
25%,33093.000000,0.000000,2.000000,2.000000,3.000000
50%,66186.000000,1.000000,4.000000,5.000000,5.000000
75%,99279.000000,1.000000,5.000000,8.000000,8.000000
max,132372.000000,1.000000,5.000000,10.000000,10.000000


removendo linhas desnecessárias e not a numbers

In [6]:
avaliacoes.drop(["original_index", "review_text_processed", "review_text_tokenized", "rating", "kfold_polarity", "kfold_rating"], axis=1)


,review_text,polarity
0,Bem macio e felpudo...recomendo. Preço imbatí...,1.0
1,Produto excepcional! recomendo!!! inovador e ...,1.0
2,recebi o produto antes do prazo mas veio com d...,0.0
3,Bom custo beneficio. Adequado para pessoas que...,1.0
4,Além de higiênico tem o tamanho ideal. Só falt...,NaN
...,...,...
132368,SMARTHPHONE PERFEITO MUITO BOM E A ECONO...,1.0
132369,Eu estou muito chateado com isso gostaria que ...,0.0
132370,"Vale a pena pelo preço oferecido, qualidade a ...",NaN
132371,O produto anunciado no site dizia ser 100% al...,0.0


In [7]:
avaliacoes.dropna(inplace=True,axis = 0)


In [8]:
avaliacoes.polarity.value_counts()

,count
polarity,
1.0,80300
0.0,35758


# Stemming RSLP

In [16]:
#remocao de stop words
from matplotlib import pyplot as plt
from nltk import tokenize, download
from nltk.corpus import stopwords
from wordcloud import WordCloud
from string import punctuation

processed_words = list()
brazilian_stop_words = stopwords.words('portuguese')
stemmer = nltk.RSLPStemmer()
pontuction_text = list()
for p in punctuation:
  pontuction_text.append(p)

brazilian_stop_words_pontuation = brazilian_stop_words + pontuction_text

for text in avaliacoes.review_text:
  new_text = list()
  tokens = tokenize.WhitespaceTokenizer().tokenize(text)
  for word in tokens:
    if word not in brazilian_stop_words_pontuation:
      #stemming cleaned words
      new_text.append(stemmer.stem(word))
  processed_words.append(' '.join(new_text))

avaliacoes["review_text_processed"] = processed_words


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
avaliacoes_tfidf_train = tfidf.fit_transform(avaliacoes["review_text_processed"])

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(avaliacoes_tfidf_train,avaliacoes.polarity, stratify=avaliacoes.polarity, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression()
regressor.fit(x_train, y_train)
accurancy = regressor.score(x_test, y_test)
print(accurancy)


0.9361967947613303


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

frases = avaliacoes.review_text_processed.tolist()
tfidf = TfidfVectorizer(lowercase=False, max_features=100)
caracteristicas = tfidf.fit_transform(frases)

pd.DataFrame(
    caracteristicas.todense(),
    columns=tfidf.get_feature_names_out()
)

,ach,ador,agor,aind,algum,além,americ,americanas,ant,aparelh,...,tv,um,us,uso,val,vei,vend,vez,vou,ótim
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.363338,0.0,0.0,0.0,0.316177,0.0,0.278150,0.000000,...,0.0,0.0,0.382617,0.000000,0.0,0.317131,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.692332,...,0.0,0.0,0.000000,0.342088,0.0,0.000000,0.000000,0.000000,0.000000,0.246515
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.419122,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116053,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.377770,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.356516
116054,0.0,0.0,0.000000,0.0,0.0,0.0,0.187378,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.232804,0.000000,0.000000,0.000000
116055,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.279735,0.000000
116056,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [33]:
tfidf = TfidfVectorizer(lowercase=False, ngram_range = (1,2))
vetor_tfidf = tfidf.fit_transform(avaliacoes.review_text_processed)
treino, teste, classe_treino, classe_teste = train_test_split(vetor_tfidf,
                                                              avaliacoes.polarity,
                                                              random_state = 42)

regressao_logistica = LogisticRegression()
regressao_logistica.fit(treino, classe_treino)
acuracia_tfidf_ngrams = regressao_logistica.score(teste, classe_teste)
acuracia_tfidf_ngrams

0.9376184732035154